In [1]:
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModel, AutoTokenizer
import torch
processor = AutoProcessor.from_pretrained("/home/user/wyf/train_siglip_from_scratch/vit-base-patch16-224")
tokenizer = AutoTokenizer.from_pretrained('/home/user/wyf/chinese-roberta-wwm-ext')
model = AutoModel.from_pretrained('/home/user/wyf/train_siglip_from_scratch/outputs')

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
/home/user/anaconda3/envs/wyf/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at /home/user/wyf/train_siglip_from_scratch/outputs were not used when initializing SiglipModel: ['b', 't', 'text_model.embeddings.LayerNorm.bias', 'text_model.embeddings.LayerNorm.weight', 'text_model.embeddings.position_embeddings.weight', 'text_model.embeddings.token_type_embeddings.weight', 'text_model.embeddings.wor

In [4]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
# image = Image.open('/home/user/wyf/train_multimodal_from_scratch/test_images/th1.png')
texts = ["两只猫在玩耍", '一颗树']
# important: we pass `padding=max_length` since the model was trained with this
input_ids = tokenizer(texts, padding="max_length", return_tensors="pt", max_length=64)['input_ids']
attention_mask = tokenizer(texts, padding="max_length", return_tensors="pt", max_length=64)['attention_mask']
pixel_values = processor(images=image, return_tensors="pt")['pixel_values']
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values)

logits_per_image = outputs.logits_per_image
probs = torch.sigmoid(logits_per_image) # these are the probabilities
print(probs)
print(f"{probs[0][0]:.1%} that image 0 is '{texts[0]}'")
print(f"{probs[0][1]:.1%} that image 0 is '{texts[1]}'")

tensor([[0.5025, 0.5097]])
50.2% that image 0 is '两只猫在玩耍'
51.0% that image 0 is '一颗树'
